![Crisp](img/logo.png)
# Crisp basics

This page shows you how to get started with Crisp blueprints

[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-orange?logo=google-colab&style=for-the-badge)](https://colab.research.google.com/github/gocrisp/blueprints/blob/main/notebooks/crisp_basics.ipynb)
[![Open in Vertex AI](https://img.shields.io/badge/Open%20in-Vertex%20AI%20Workbench-brightgreen?logo=google-cloud&style=for-the-badge)](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/gocrisp/blueprints/main/notebooks/crisp_basics.ipynb)
[![Open in Databricks](https://img.shields.io/badge/databricks-red?logo=databricks&style=for-the-badge)](https://www.databricks.com/try-databricks)
[![View on GitHub](https://img.shields.io/badge/View%20on-GitHub-lightgrey?logo=github&style=for-the-badge)](https://github.com/gocrisp/blueprints/blob/main/notebooks/crisp_basics.ipynb)



## Set the required environment variables

In [1]:
import os

os.environ["ACCOUNT_ID"] = "80548"
os.environ["CONNECTOR_ID"] = "3378"  # optional

### Run Crisp common

This notebook uses the [crisp_common.ipynb](./crisp_common.ipynb) notebook to load the common functions and variables. The `crisp_common.ipynb` notebook contains the common functions and variables that are used across the Crisp notebooks.

In [ ]:
import os

if not os.path.exists("crisp_common.ipynb"):
    print("Downloading crisp_common.ipynb")
    !wget https://raw.githubusercontent.com/gocrisp/blueprints/main/notebooks/crisp_common.ipynb -O crisp_common.ipynb
else:
    print("crisp_common.ipynb already exists")

%run crisp_common.ipynb

## Load data into a DataFrame

The following code loads data from a table into a DataFrame. 
Variables `project`, `dataset` are taken from the `crisp_common.ipynb` notebook. but you can also set them here.

In [3]:
# project = 'your_project'
# dataset = 'your_dataset'
table = "normalized_unfi_fact_sales"

print("Loading data from {}.{}.{}".format(project, dataset, table))

Loading data from crisp-frontier-dev.analytics_blueprints_80548.normalized_unfi_fact_sales


In [4]:
%%load df
SELECT * FROM `{project}`.`{dataset}`.`{table}` LIMIT 10

In [5]:
display(df)

,account_id,connector_config_id,channel_id,date_key,dc_id,is_first_purchase,product_id,sales_amount,sales_id,sales_quantity,store_id,uom_id,whole_sale_price
0,80548,82,9221995035584428660,2020-04-05,3475258104728469248,False,4001433058368439128,39.83,2016740781568767598,2.0,3775738315386423810,7490622694260177996,71.10
1,80548,82,9221995035584428660,2020-05-03,3475258104728469248,False,4065777199066685072,40.16,8418246900861692640,2.0,3775738315386423810,7490622694260177996,69.76
2,80548,82,9221995035584428660,2020-03-08,3475258104728469248,False,6079228713510413679,23.75,8084158043312151039,2.0,3775738315386423810,7490622694260177996,42.38
3,80548,82,9221995035584428660,2020-03-01,3475258104728469248,True,2882915166545356769,39.83,2564654221828368863,2.0,3775738315386423810,7490622694260177996,71.10
4,80548,82,9221995035584428660,2020-06-07,3475258104728469248,False,7005387553611591933,32.66,2782437254778047654,2.0,3775738315386423810,7490622694260177996,58.47
5,80548,82,9221995035584428660,2020-03-08,3475258104728469248,False,4065777199066685072,40.16,7749986086792519715,2.0,3775738315386423810,7490622694260177996,69.76
6,80548,82,6783875957240731496,2020-08-02,3475258104728469248,False,6079228713510413679,36.93,5560291402395423046,2.0,4143340026718314243,7490622694260177996,42.38
7,80548,82,6783875957240731496,2020-05-10,3475258104728469248,False,6079228713510413679,40.95,9065978831707661471,2.0,3651889994219835910,7490622694260177996,42.38
8,80548,82,8802252399211565501,2020-04-05,3475258104728469248,False,6079228713510413679,28.89,4385233044043201791,2.0,8158541374731853063,7490622694260177996,42.38
9,80548,82,8802252399211565501,2020-03-08,3475258104728469248,True,6079228713510413679,34.48,8547972509903588845,2.0,8158541374731853063,7490622694260177996,42.38


## Save DataFrame to table

In [6]:
%%save {project}.{dataset}.normalized_unfi_fact_sales_sample
df

## Save query results to table

In [7]:
%%save {project}.{dataset}.normalized_unfi_fact_sales_sample2
SELECT * FROM `{project}`.`{dataset}`.`normalized_unfi_fact_sales` LIMIT 10